# Digital Twin Fairways
WIP to use the OpenTNSim engine

In [1]:
import datetime
import json
import logging
import time

import dtv_backend.berthing

# the simpy processes and objects
import dtv_backend.compat

# library to load the fairway information network
import dtv_backend.fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate
import geojson
import networkx as nx
import pandas as pd
import shapely
import simpy
from networkx.readwrite import json_graph

# opentnsim import
from opentnsim import core

# reload for debugging purposes
%load_ext autoreload
%autoreload 2

logger = logging.getLogger("notebook")

/Users/baart_f/.virtualenvs/main/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning:

The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



### Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [3]:
# example input
with open("../../dtv_backend/tests/2022-09-05.geojson") as f:
    config = geojson.load(f)

### Simulate with ``v3_run``

In [4]:
def v3_dtv_backend_simulate(config):
    result = dtv_backend.simulate.v3_run(config)
    env = result['env']
    logbook = result['operator'].logbook
    log_df = pd.DataFrame(logbook)
    log_json = dtv_backend.postprocessing.log2json(log_df)
    
    response = {
        "log": log_json,
        "config": config,
        "env": {
            "epoch": env.epoch.timestamp(),
            "epoch_iso": env.epoch.isoformat(),
            "now": env.epoch,
            "now_iso": datetime.datetime.fromtimestamp(env.now).isoformat(),
        },
    }
    return response
    

In [5]:
resp = v3_dtv_backend_simulate(config)

/Users/baart_f/src/digitaltwin-waterway/dtv_backend/dtv_backend/fis.py:111: ShapelyDeprecationWarning:

The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.



8862214
8862214


In [5]:
config['fleet'][0]['properties']

{'CEMT-class': 'Va',
 'RWS-class': 'M8',
 'Vessel type': 'Motorvessel',
 'Description (English)': 'Large Rhine vessel',
 'Description (Dutch)': 'Groot Rijnschip',
 'Beam [m]': 11.4,
 'Beam 10% percentile [m]': 10.6,
 'Beam 50% percentile [m]': 11.45,
 'Beam 90% percentile [m]': 11.45,
 'Length [m]': 110,
 'Length 10% percentile [m]': 93,
 'Length 50% percentile [m]': 110,
 'Length 90% percentile [m]': 110,
 'Draught loaded [m]': 3.5,
 'Draught average [m]': 3.14,
 'Draught empty [m]': 1.4,
 'Height average [m]': 6.21,
 'Load Weight average [ton]': 2286,
 'Load weight maximum [ton]': 2689,
 'speed': 3,
 'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQRvKRniAxUXUWzmByw7CRFYD5fTqOtFTDVkw&usqp=CAU',
 'capacity': 1600,
 'name': 'Large Rhine vessel',
 'count': 1}